####  Dense optical flow, specifically the Farneback algorithm from OpenCV, is used on videos of GOES-16 satellite images to estimate cloud motion (speed) over the Atmospheric Radiation Measurement (ARM) Southern Great Plains (SGP) Central Facility. 
Optical Flow in OpenCV: https://learnopencv.com/optical-flow-in-opencv/

In [1]:
import cv2
import numpy as np

In [2]:
def dense_optical_flow(method, video_path, params=[], to_gray=True):
    
    # Read the video
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame
    ret, old_frame = cap.read()

    # Create HSV & make value a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255

    # Preprocessing for exact method
    if to_gray:
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    
    i=0
    
    while True:
        # Read the next frame
        ret, new_frame = cap.read()
        frame_copy = new_frame
        if not ret:
            break
            
        # Preprocessing for exact method
        if to_gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
            
        # Calculate Optical Flow
        flow = method(old_frame, new_frame, None, *params)

        # Encoding: convert the algorithm's output into Polar coordinates
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Use Hue and Saturation to encode the Optical Flow
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        
        # Convert HSV image into BGR for demo
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # Show video frames and optical flow
        cv2.imshow("frame", frame_copy)
        cv2.imshow("optical flow", bgr)
       
        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break
        
        # Save optical flow hsv images
        name = f'./opticalhsv_vis{i}.png'
        cv2.imwrite(name,bgr)
        
        # Select the magnitude (speed) at a single point
        #magn = mag[540, 960]
        
        # Select the magnitude (speed) for an array of pixels
        magn = mag[535:545,955:965]
        
        # Convert pixel units to m/s
        m_s = (magn / 300) * 500
        
        # Append speed in m/s
        speed_m_s.append(m_s)
        
        old_frame = new_frame 
        i+=1
    
   # Motions are averaged per pixel over the length of the video 
    speed = (sum(speed_m_s)) / i
    print(speed)

    # Averaging the motion at all the selected pixels
    print(np.mean(speed)) 

In [3]:
speed_m_s = []
method = cv2.calcOpticalFlowFarneback
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # default Farneback algorithm parameters
videopath = 'C:/Users/Margo/GOES16/GOES16_vis_17_19Z_160.mp4'

frames = dense_optical_flow(method, videopath, params, to_gray=True)

[[8.299636  7.9852433 7.692396  7.403664  7.0517364 6.5956993 6.075887
  5.5671105 5.058112  4.586263 ]
 [8.066425  7.775867  7.4887996 7.2051563 6.855211  6.42635   5.944124
  5.4724655 4.982873  4.523262 ]
 [7.8809834 7.6027737 7.314631  7.015764  6.678978  6.291532  5.853943
  5.4250145 4.9673543 4.5253143]
 [7.6611333 7.3944874 7.109695  6.822663  6.525585  6.180597  5.7773223
  5.3843026 4.9710283 4.552159 ]
 [7.411463  7.1591773 6.878802  6.6069865 6.3475285 6.0225396 5.6369514
  5.2644067 4.8746657 4.469699 ]
 [7.143177  6.9094315 6.64014   6.3779063 6.123677  5.808874  5.4354053
  5.0773463 4.704499  4.305757 ]
 [6.8283687 6.6200423 6.3760147 6.115932  5.8506484 5.554262  5.2143636
  4.890534  4.541521  4.1469107]
 [6.5504527 6.3647676 6.1379013 5.8732038 5.585811  5.2955685 4.990702
  4.7103095 4.3898697 4.0019126]
 [6.274065  6.102982  5.8860016 5.622715  5.3188014 5.015899  4.7297864
  4.4755034 4.178092  3.8002112]
 [5.7808075 5.6411953 5.45977   5.240048  4.973536  4.67713

In [4]:
cv2.destroyAllWindows()